# 06_05_neu: Image Sampler & Quick CV Lab

Ziele:
- Bestehende Demo-Bilder nutzen (oder Fallback-Synthetics)
- Schneller Image-Browser ohne Training
- Edge/Contour/ORB-Pipeline aus den neuen Notebooks wiederverwenden
- Einfache Augmentations als visuelle Vorschau
- CPU-freundlich, alles lokal/offline

## Warum dieses Notebook?
- Mini-Story: Du bereitest Bilder für ein kleines Edge-Detection-PoC vor und musst schnell prüfen, welche Schritte (Canny/Contours/ORB) auf deinem Datenstapel funktionieren.
- Ziel: Ohne Training sofort sehen, was deine Pipeline mit echten Assets macht, und schnelle Parameter-Iterationen fahren.
- Entscheidungshilfe: Wann Canny? (saubere Kanten), Wann Contours? (Objekt-Umrisse zählen), Wann ORB? (Keypoints/Matching, patentfrei).

In [1]:
# Imports & Setup
import os, glob, random
import numpy as np
import matplotlib.pyplot as plt
import cv2
from ipywidgets import interact, Dropdown, IntSlider, FloatSlider, Checkbox, fixed

plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)
random.seed(42)
print("OpenCV", cv2.__version__)


OpenCV 4.12.0


In [2]:
# Helpers: list and load images

def list_images(root="images"):
    patterns = ["*.png", "*.jpg", "*.jpeg", "*.bmp", "*.gif"]
    files = []
    for pat in patterns:
        files.extend(glob.glob(os.path.join(root, "**", pat), recursive=True))
    files = [os.path.relpath(f, root) for f in files]
    return sorted(files)


def load_image(path=None, size=(400, 400), root="images"):
    if path and path != "synthetic_stripes":
        full = os.path.join(root, path)
        if os.path.exists(full):
            img = cv2.cvtColor(cv2.imread(full), cv2.COLOR_BGR2RGB)
            if size:
                img = cv2.resize(img, size)
        else:
            print(f"Missing file: {full}, using synthetic instead")
            img = None
    else:
        img = None
    if img is None:
        xs = np.linspace(0, 1, size[0])
        ys = np.linspace(0, 1, size[1])
        xv, yv = np.meshgrid(xs, ys)
        img = np.stack([
            (xv * 255).astype(np.uint8),
            (yv * 255).astype(np.uint8),
            ((xv > 0.5) ^ (yv > 0.5)).astype(np.uint8) * 255,
        ], axis=-1)
    if img.dtype != np.uint8:
        img = np.clip(img, 0, 255).astype(np.uint8)
    return img

images = list_images()
options = ["synthetic_stripes"] + images
print(f"Found {len(images)} image(s) under images/; defaulting to synthetic if empty.")


Found 32 image(s) under images/; defaulting to synthetic if empty.


In [3]:
# Browse a sample image

@interact(img=Dropdown(options=options, description="image"))
def show_image(img="synthetic_stripes"):
    arr = load_image(img)
    plt.figure(figsize=(5,5))
    plt.imshow(arr)
    plt.axis("off")
    plt.title(img)
    plt.show()

interactive(children=(Dropdown(description='image', options=('synthetic_stripes', 'Faltung1.png', 'Faltung2.pn…

In [4]:
# Edge detectors (reuse helpers)

def to_gray_u8(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) if img.ndim == 3 else img
    if gray.dtype != np.uint8:
        gray = np.clip(gray, 0, 255).astype(np.uint8)
    return gray


def run_edges(img, low=80, high=150, ksize=3):
    gray = to_gray_u8(img)
    canny = cv2.Canny(gray, low, high)
    sobelx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=ksize)
    sobely = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=ksize)
    mag = np.sqrt(sobelx**2 + sobely**2)
    mag = np.clip(mag / (mag.max() + 1e-6) * 255, 0, 255).astype(np.uint8)
    lap = cv2.Laplacian(gray, cv2.CV_32F)
    lap = np.clip((lap - lap.min()) / (lap.max() - lap.min() + 1e-6) * 255, 0, 255).astype(np.uint8)
    fig, axes = plt.subplots(1, 4, figsize=(12, 3))
    for ax, img_, title in zip(axes, [gray, canny, mag, lap], ["Gray", "Canny", "Sobel mag", "Laplacian"]):
        ax.imshow(img_, cmap="gray")
        ax.set_title(title)
        ax.axis("off")
    plt.tight_layout()
    plt.show()


@interact(
    img=Dropdown(options=options, description="image"),
    low=IntSlider(80, 0, 200, 10, description="low"),
    high=IntSlider(150, 50, 300, 10, description="high"),
    ksize=IntSlider(3, 3, 7, 2, description="ksize")
)
def _tune_edges(img="synthetic_stripes", low=80, high=150, ksize=3):
    if high <= low:
        high = low + 10
    arr = load_image(img)
    run_edges(arr, low, high, int(ksize))


interactive(children=(Dropdown(description='image', options=('synthetic_stripes', 'Faltung1.png', 'Faltung2.pn…

In [5]:
# Contours

def detect_contours(img, thresh1=80, thresh2=150):
    gray = to_gray_u8(img)
    edges = cv2.Canny(gray, thresh1, thresh2)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    result = img.copy()
    cv2.drawContours(result, contours, -1, (255, 0, 0), 2)
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(edges, cmap="gray")
    plt.title("Canny")
    plt.axis("off")
    plt.subplot(1, 2, 2)
    plt.imshow(result)
    plt.title(f"Contours: {len(contours)}")
    plt.axis("off")
    plt.tight_layout()
    plt.show()
    return contours


@interact(
    img=Dropdown(options=options, description="image"),
    thresh1=IntSlider(80, 0, 200, 10, description="th1"),
    thresh2=IntSlider(150, 50, 300, 10, description="th2"),
)
def _tune_contours(img="synthetic_stripes", thresh1=80, thresh2=150):
    if thresh2 <= thresh1:
        thresh2 = thresh1 + 10
    arr = load_image(img)
    detect_contours(arr, thresh1, thresh2)


interactive(children=(Dropdown(description='image', options=('synthetic_stripes', 'Faltung1.png', 'Faltung2.pn…

In [6]:
# ORB keypoints

def run_orb(img, n_features=300):
    orb = cv2.ORB_create(nfeatures=n_features)
    gray = to_gray_u8(img)
    kps, desc = orb.detectAndCompute(gray, None)
    vis = cv2.drawKeypoints(img, kps, None, color=(0, 255, 0), flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)
    plt.figure(figsize=(6, 4))
    plt.imshow(vis)
    plt.axis("off")
    plt.title(f"ORB keypoints: {len(kps)}")
    plt.show()
    return kps, desc


@interact(
    img=Dropdown(options=options, description="image"),
    n_features=IntSlider(200, 50, 500, 50, description="features"),
)
def _tune_orb(img="synthetic_stripes", n_features=200):
    arr = load_image(img)
    run_orb(arr, n_features)


interactive(children=(Dropdown(description='image', options=('synthetic_stripes', 'Faltung1.png', 'Faltung2.pn…

In [ ]:
# Simple augmentation preview (no training)

def augment(img, flip_h=False, flip_v=False, rotate_deg=0, brightness=0, contrast=1.0):
    aug = img.copy()
    if flip_h:
        aug = cv2.flip(aug, 1)
    if flip_v:
        aug = cv2.flip(aug, 0)
    if rotate_deg != 0:
        h, w = aug.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, rotate_deg, 1.0)
        aug = cv2.warpAffine(aug, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
    aug = cv2.convertScaleAbs(aug, alpha=float(contrast), beta=float(brightness))
    return aug


@interact(
    img=Dropdown(options=options, description="image"),
    flip_h=Checkbox(False, description="flip_h"),
    flip_v=Checkbox(False, description="flip_v"),
    rotate_deg=IntSlider(0, -25, 25, 1, description="rotate"),
    brightness=IntSlider(0, -50, 50, 5, description="bright"),
    contrast=FloatSlider(value=1.0, min=0.5, max=1.5, step=0.05, description="contrast"),
)
def _preview_aug(img="synthetic_stripes", flip_h=False, flip_v=False, rotate_deg=0, brightness=0, contrast=1.0):
    arr = load_image(img)
    aug = augment(arr, flip_h, flip_v, rotate_deg, brightness, contrast)
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(arr)
    plt.title("Original")
    plt.axis("off")
    plt.subplot(1, 2, 2)
    plt.imshow(aug)
    plt.title("Augmented")
    plt.axis("off")
    plt.tight_layout()
    plt.show()


TypeError: _Float.__init__() takes from 1 to 2 positional arguments but 5 were given

## Tipps
- Läuft komplett offline; wenn `images/` fehlt, werden synthetische Streifen genutzt.
- Für schnelle Läufe Bildgröße bei Bedarf in `load_image(size=...)` weiter verkleinern.
- Die Widgets nutzen dieselben Kern-Schritte wie in den neuen Notebooks, aber ohne Training/Weights.

## Selbstcheck
- Welche drei Schritte bietet das Notebook an und wann würdest du jeden verwenden?
- Was passiert, wenn `images/` fehlt?
- Wie veränderst du die Performance auf einer langsamen CPU?

## Troubleshooting
- Widgets rendern nicht: Stelle sicher, dass `ipywidgets` installiert/aktiviert ist und Kernel neu starten.
- Canny/Contours leer: Schwellenwerte senken oder Bildgröße reduzieren.
- ORB liefert wenige Keypoints: `n_features` erhöhen und auf Graustufen achten (kontrastreiche Bilder).
- Augmentation verzerrt: Rotationswinkel kleiner wählen oder Border-Mode auf REFLECT lassen.